<a href="https://colab.research.google.com/github/qoolseal/chemistry-telegram-bot/blob/main/test_bott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install googletrans==3.1.0a0

In [ ]:
import googletrans
from googletrans import Translator as translator

result = translator.translate('Mitä sinä teet')
result.dest

In [ ]:
pip install python-telegram-bot

In [ ]:
import requests
from bs4 import BeautifulSoup

from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler
updater = Updater('<token>', use_context=True)
      # telegram.ext.Updater получает все обновления/сообщения в автоматическом режиме.
dispatcher = updater.dispatcher
      # в объекте Dispatcher можно зарегистрировать обработчики разных типов,
      # которые будут сортировать полученные объектом Updater сообщения
from telegram.ext import MessageHandler, Filters  # модули необходимые для обработки сообщений
import logging  # Чтобы знать, когда и почему что-то не работает должным образом,
                # настроим модуль ведения журнала логов
logging.basicConfig(format='%(levelname)s - %(message)s', level =logging.DEBUG)
logger = logging.getLogger(__name__)

def pred(update, context):
  '''функция, предсказывающая продукты реакции(берётся с сайта https://chemequations.com/ru/)'''
  if context.args:
    pr = context.args
    new = str(pr[0])
    for i in range(1,len(pr)):
      new+= '+%2B+' + str(pr[i])
    u = f'https://chemequations.com/ru/?s={new}&ref=input'
    response = requests.get(u)
    soup = BeautifulSoup(response.text, 'lxml')
    quotes = soup.find_all('span', class_ = 'group')
    soup1 = soup.find_all('a')
    href = soup1[11].get('href')
    try:
      if soup.find('caption').text == 'Существует несколько решений:':
        context.bot.send_message(chat_id=update.effective_chat.id,
                                text='Существует несколько решений. Вот самое популярное:')
        u = 'https://chemequations.com/ru/' + str(href)
        response = requests.get(u)
        soup = BeautifulSoup(response.text, 'lxml')
        quotes = soup.find_all('span', class_ = 'group')
    except AttributeError:
      quotes = soup.find_all('span', class_ = 'group')
    ur = []
    for q in quotes:
      a = q.text
      if a.startswith('(') and a.endswith(')'):
        continue
      elif a in ur:
        continue
      else:
        ur.append(a)
    if len(ur) == len(pr):
      context.bot.send_message(chat_id=update.effective_chat.id,
                                text='Не нашлось ответа. Попробуйте другую реакцию')
    else:
      ans = str(ur[0])
      ur = ur[1:]
      for i in range(len(ur)):
        if ur[i] in pr:
          ans += ' + ' + str(ur[i])
        #context.bot.send_message(chat_id=update.effective_chat.id,
                                #text=f'{ans}\n{pr}')
        if ur[i] not in pr:
          ans += ' => ' + str(ur[i])
          n = ur.index(ur[i])
          break

      if len(ur) != n:
        ur = ur[int(n + 1):]
        for i in range(len(ur)):
          ans += ' + ' + str(ur[i])

      context.bot.send_message(chat_id=update.effective_chat.id, text=ans)
      context.bot.send_message(chat_id=update.effective_chat.id, text='А теперь можете уравнять сами или при помощи команды /eq')
  else:
    context.bot.send_message(chat_id=update.effective_chat.id,
                                text='Нет аргументов')
    context.bot.send_message(chat_id=update.effective_chat.id,
                                text='Отправьте: /pred и реагенты через пробел\nНапример, /pred H2SO4 NH3')

updater.dispatcher.add_handler(CommandHandler('pred', pred))


updater.start_polling()# говорим объекту updater слушать сервера telegram
updater.idle()# обработчик нажатия Ctrl+C для остановки
updater.stop()